In [1]:
import requests
import csv
import time
import os
from datetime import datetime
from typing import Dict
from dotenv import load_dotenv

In [41]:
date_suffix=datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
date_suffix

'2025_06_03_20_40_59'

In [2]:
# Load .env file from a specific path
dotenv_path = "E:/tolls/.env"  # <-- Replace with the actual path
load_dotenv(dotenv_path)

API_KEY = os.getenv("GOOGLE_API_KEY")
if not API_KEY:
    raise EnvironmentError("GOOGLE_API_KEY not set in the .env file.")

In [3]:
# Google Places API configuration
BASE_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [19]:
# Keyword to search
KEYWORD = "EV charging point"  # Change as needed

In [48]:
# f"{KEYWORD.replace(' ','_')}_nearby_{date_suffix}.csv"

In [49]:
# f"{KEYWORD.replace(' ','_')}_log_{date_suffix}.csv"

In [4]:
# Output files
OUTPUT_CSV = f"{KEYWORD.replace(' ','_')}_nearby_{date_suffix}.csv"
LOG_FILE = f"{KEYWORD.replace(' ','_')}_log_{date_suffix}.csv"

In [6]:
# API Call Counter
api_call_count = 0

In [7]:
# CSV Headers
CSV_HEADERS = ["place_id", "name", "formatted_address", "latitude", "longitude"]

In [8]:
# Ensure CSV file exists with headers
def init_csv(file_path: str):
    if not os.path.exists(file_path):
        with open(file_path, mode='w', newline='', encoding='utf-8') as f:
            writer = csv.DictWriter(f, fieldnames=CSV_HEADERS)
            writer.writeheader()

In [9]:
# Write log entry
def write_log(message: str):
    with open(LOG_FILE, mode='a', encoding='utf-8') as f:
        f.write(f"{datetime.now().isoformat()} - {message}\n")

In [10]:
# Save place record
existing_place_ids = set()
def save_to_csv(record: Dict):
    if record["place_id"] in existing_place_ids:
        return
    existing_place_ids.add(record["place_id"])
    with open(OUTPUT_CSV, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=CSV_HEADERS)
        writer.writerow(record)

In [11]:
# Avoid duplicate API calls
called_coords = set()
def is_duplicate_call(lat: float, lng: float) -> bool:
    key = (round(lat, 4), round(lng, 4))
    if key in called_coords:
        return True
    called_coords.add(key)
    return False

In [12]:
# Fetch places for a point
def fetch_places(lat: float, lng: float):
    global api_call_count
    if is_duplicate_call(lat, lng):
        return

    write_log(f"API call #{api_call_count + 1} at lat: {lat}, lng: {lng}, keyword: {KEYWORD}")

    next_page_token = None
    while True:
        params = {
            "location": f"{lat},{lng}",
            "radius": 50000,
            "keyword": KEYWORD,
            "key": API_KEY
        }
        if next_page_token:
            params['pagetoken'] = next_page_token
            time.sleep(2)

        response = requests.get(BASE_URL, params=params)
        api_call_count += 1
        data = response.json()
        for result in data.get("results", []):
            record = {
                "place_id": result.get("place_id", ""),
                "name": result.get("name", ""),
                "formatted_address": result.get("vicinity", ""),
                "latitude": result.get("geometry", {}).get("location", {}).get("lat", 0),
                "longitude": result.get("geometry", {}).get("location", {}).get("lng", 0)
            }
            save_to_csv(record)

        next_page_token = data.get("next_page_token")
        if not next_page_token:
            break

In [13]:
# Generate lat/lng grid for a state
def generate_grid_for_state(center_lat, center_lng, delta=2.0, step=1.0):
    points = []
    min_lat = center_lat - delta
    max_lat = center_lat + delta
    min_lng = center_lng - delta
    max_lng = center_lng + delta

    lat = min_lat
    while lat <= max_lat:
        lng = min_lng
        while lng <= max_lng:
            points.append((lat, lng))
            lng += step
        lat += step
    return points

In [14]:
# Main driver for a single state
def main_for_state(state_name: str, center_lat: float, center_lng: float):
    global api_call_count
    init_csv(OUTPUT_CSV)
    write_log(f"Starting data collection for {state_name}")
    points = generate_grid_for_state(center_lat, center_lng)
    for lat, lng in points:
        fetch_places(lat, lng)
        time.sleep(1)
    
    cost_estimate = round(api_call_count / 1000 * 32.0, 2)
    write_log(f"Finished data collection for {state_name}")
    write_log(f"Total API calls: {api_call_count}")
    write_log(f"Estimated API cost (USD): ${cost_estimate}")

In [29]:
indian_states_and_uts = [
    # ("Andhra Pradesh", 15.9129, 79.7400),
    # ("Arunachal Pradesh", 28.2180, 94.7278),
    ("Assam", 26.2006, 92.9376),
    ("Bihar", 25.0961, 85.3131),
    ("Chhattisgarh", 21.2787, 81.8661),
    ("Goa", 15.2993, 74.1240),
    ("Gujarat", 22.2587, 71.1924),
    ("Haryana", 29.0588, 76.0856),
    ("Himachal Pradesh", 31.1048, 77.1734),
    ("Jharkhand", 23.6102, 85.2799),
    # ("Karnataka", 15.3173, 75.7139),
    ("Kerala", 10.8505, 76.2711),
    ("Madhya Pradesh", 22.9734, 78.6569),
    ("Maharashtra", 19.7515, 75.7139),
    ("Manipur", 24.6637, 93.9063),
    ("Meghalaya", 25.4670, 91.3662),
    ("Mizoram", 23.1645, 92.9376),
    ("Nagaland", 26.1584, 94.5624),
    ("Odisha", 20.9517, 85.0985),
    ("Punjab", 31.1471, 75.3412),
    ("Rajasthan", 27.0238, 74.2179),
    ("Sikkim", 27.5330, 88.5122),
    ("Tamil Nadu", 11.1271, 78.6569),
    ("Telangana", 18.1124, 79.0193),
    ("Tripura", 23.9408, 91.9882),
    ("Uttar Pradesh", 26.8467, 80.9462),
    ("Uttarakhand", 30.0668, 79.0193),
    ("West Bengal", 22.9868, 87.8550),

    # Union Territories
    ("Andaman and Nicobar Islands", 11.7401, 92.6586),
    ("Chandigarh", 30.7333, 76.7794),
    ("Dadra and Nagar Haveli and Daman and Diu", 20.1809, 73.0169),
    ("Delhi", 28.6139, 77.2090),
    ("Jammu and Kashmir", 33.7782, 76.5762),
    ("Ladakh", 34.2268, 77.5619),
    ("Lakshadweep", 10.3280, 72.7846),
    ("Puducherry", 11.9416, 79.8083)
]

In [30]:
# for state in indian_states_and_uts:
#     print(state,type(state))

In [31]:
if __name__ == "__main__":
    for state in indian_states_and_uts:
        print(state)
        main_for_state(state[0],state[1],state[2])


('Assam', 26.2006, 92.9376)
('Bihar', 25.0961, 85.3131)
('Chhattisgarh', 21.2787, 81.8661)
('Goa', 15.2993, 74.124)
('Gujarat', 22.2587, 71.1924)
('Haryana', 29.0588, 76.0856)
('Himachal Pradesh', 31.1048, 77.1734)
('Jharkhand', 23.6102, 85.2799)
('Kerala', 10.8505, 76.2711)
('Madhya Pradesh', 22.9734, 78.6569)
('Maharashtra', 19.7515, 75.7139)
('Manipur', 24.6637, 93.9063)
('Meghalaya', 25.467, 91.3662)
('Mizoram', 23.1645, 92.9376)
('Nagaland', 26.1584, 94.5624)
('Odisha', 20.9517, 85.0985)
('Punjab', 31.1471, 75.3412)
('Rajasthan', 27.0238, 74.2179)
('Sikkim', 27.533, 88.5122)
('Tamil Nadu', 11.1271, 78.6569)
('Telangana', 18.1124, 79.0193)
('Tripura', 23.9408, 91.9882)
('Uttar Pradesh', 26.8467, 80.9462)
('Uttarakhand', 30.0668, 79.0193)
('West Bengal', 22.9868, 87.855)
('Andaman and Nicobar Islands', 11.7401, 92.6586)
('Chandigarh', 30.7333, 76.7794)
('Dadra and Nagar Haveli and Daman and Diu', 20.1809, 73.0169)
('Delhi', 28.6139, 77.209)
('Jammu and Kashmir', 33.7782, 76.5762)
('